In [1]:
from sympy.abc import x,y,z
from sympy import lambdify,diff, sqrt, simplify
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from scipy.optimize import minimize
from numba import jit

def compute_jet(ff, xs, ys, zs):
    f = lambdify([x,y,z],ff)
    fx = lambdify([x,y,z], diff(ff,x))
    fy = lambdify([x,y,z], diff(ff,y))
    fz = lambdify([x,y,z], diff(ff,z))
    fxs = fx(xs,ys,zs)
    fys = fy(xs,ys,zs)
    fzs = fz(xs,ys,zs)
    ns = np.sqrt(fxs**2 + fys**2 + fzs**2)
    return f(xs,ys,zs), fxs/ns, fys/ns, fzs/ns

def grid_xyz(box, res):
    xmin, xmax, ymin, ymax, zmin, zmax = box
    xs = np.linspace(xmin, xmax, res)
    ys = np.linspace(ymin, ymax, res)
    zs = np.linspace(zmin, zmax, res)
    X,Y,Z = np.meshgrid(xs, ys, zs)
    X = X.flatten()
    Y = Y.flatten()
    Z = Z.flatten()
    return X, Y, Z

# Sphere triangulation

In [2]:

res = 3
# Load the data
base = "../SphereTriangulation/"
sph_fs = np.loadtxt(base+'triangles%s.txt'%res).astype(int)
sph_vs = np.loadtxt(base+'vertices%s.txt'%res)

print("Spherical points count: %s"%sph_vs.shape[0])

# plot shperical triangles
fig = go.Figure()
fig.add_trace(go.Mesh3d(x=sph_vs[:,0], y=sph_vs[:,1], z=sph_vs[:,2], i=sph_fs[:,0], j=sph_fs[:,1], k=sph_fs[:,2], color='gray', opacity=1.0, flatshading=True))
fig.add_trace(go.Scatter3d(x=sph_vs[:,0], y=sph_vs[:,1], z=sph_vs[:,2], mode='markers', marker=dict(size=2, color='red')))
fig.update_layout(width=600, height=600)
fig.show()

Spherical points count: 642


# Surface

In [3]:
R = 2
r = 1
ax = .1
ay = .1
eq = (sqrt(x**2+y**2)- R)**2 + (z-ax*x**2+ay*y**2)**2 - r**2
box = [-3,3,-3,3,-1.5,1.5]
X, Y, Z = grid_xyz(box,100)
f, nx, ny, nz = compute_jet(eq, X, Y, Z)

cf = np.abs(f) < 0.05
surface_trace = go.Scatter3d(x=X[cf], y=Y[cf], z=Z[cf], mode='markers', marker=dict(size=1, color='orange'))

fig = go.Figure()
fig.add_trace(surface_trace)
fig.update_layout(width=800, height=800)
fig.show()

# Inverse Gauss map for points

In [4]:
inv_points = []

x_inv = []
y_inv = []
z_inv = []
for p in sph_vs:
    cn = np.sqrt((nx-p[0])**2 + (ny-p[1])**2 + (nz-p[2])**2) < 0.05
    cfn = cf & cn
    x = X[cfn]
    y = Y[cfn]
    z = Z[cfn]
    x_inv.extend(x)
    y_inv.extend(y)
    z_inv.extend(z)
x_inv = np.array(x_inv)
y_inv = np.array(y_inv)
z_inv = np.array(z_inv)



fig = go.Figure()
#fig.add_trace(surface_trace)
fig.add_trace(go.Scatter3d(x=x_inv, y=y_inv, z=z_inv, mode='markers', marker=dict(size=1, color='red')))
fig.update_layout(width=800, height=800)
fig.show()

In [5]:
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2
res = minimize(fun, (2, 0))
res.x

array([0.99999997, 2.5       ])